# LOAD DATA FILE GENERATED BY STEP

## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "pandas\..*" )

In [ ]:
import os
import datetime
import importlib
import utils as ut
import numpy as np
import pandas as pd

In [ ]:
exp_type = 'E6'

## Path Setting

In [ ]:
submission_version = False

if submission_version:
    DS_path_main = '\notebooks\dataset\mvtec_anomaly_detection'
else:
    DS_path_main = 'D:\DS\AD\mvtec_anomaly_detection'

DS_name = 'hazelnut'
DS_path = os.path.join(DS_path_main,DS_name)
codes_path = os.getcwd()
results_path=os.path.join(codes_path,'results')
results_subs        = os.path.join(results_path,    'test_results')
path_csv    = os.path.join(results_path,     'csv')

# Default Parameters

In [ ]:
epochs = 30000
num_explained_classes = 1
batch_size = 16
image_size=[128, 128]
latent_dim=32
mask_type          = 'blend_2'
postfix_csv = f'{DS_name}_{epochs}_9'

# Read Pre-Computed CSV 

In [ ]:
path_csv    = '../results_logits_with_auc_clipped/'

In [ ]:
csv_filename = f'{path_csv}/csv_exp_{postfix_csv}_BPT.csv'
print(csv_filename,' *** ',os.path.exists(csv_filename))

In [ ]:
df = pd.read_csv(csv_filename)
df.head(3)

In [ ]:
print('Total Time: \t\t',str(datetime.timedelta(seconds=np.sum(df.time_total))))

In [ ]:
importlib.reload(ut)   
train_dir,gtruth_dir,test_dir = ut.Data.get_subdata(DS_path=DS_path)
anomaly_types    = sorted(os.listdir(test_dir))
anomaly_types_gt = sorted(os.listdir(gtruth_dir))
anomaly_types,anomaly_types_gt = ut.Data.get_anomaly_types(test_dir=test_dir,dataset=DS_name,gt_dir=gtruth_dir)
image_counter_cls = []
for i in range(len(anomaly_types)):
    image_counter_cls.extend([[anomaly_types[i],len(os.listdir(os.path.join(test_dir,anomaly_types[i])))]])
train_dir,gtruth_dir,test_dir = ut.Data.get_subdata(DS_path=DS_path)

augmentation_target = 'custom' # 'medium', 'full' minimal , custom
train_generator,test_generator, gtruth_generator= ut.Data2.load_data(train_dir = train_dir,
                                                                     test_dir = test_dir,
                                                                     gtruth_dir = gtruth_dir,
                                                                     dataset=DS_name,
                                                                     classes = anomaly_types,
                                                                     augmentation_target=augmentation_target,
                                                                    )

In [ ]:
image_counter = {class_name: len(os.listdir(os.path.join(test_dir, class_name))) for class_name in test_generator.class_indices}
print(image_counter)

min_image_count = min(image_counter.items(), key=lambda x: x[1])[1]
min_image_class = min(image_counter.items(), key=lambda x: x[1])[0]
print(min_image_class,min_image_count)
from collections import Counter
Counter(test_generator.classes)
lbls_cls = list(image_counter.keys())
count_cls = list(image_counter.values())
lbls_cls,count_cls

# Read Pre-Computed CSV File for Anomaly Scores

In [ ]:
postfix_csv = f'{DS_name}_{epochs}_AS'
ascsv_filename = f'{path_csv}/csv_exp_{postfix_csv}.csv'
df_as= pd.read_csv(ascsv_filename)

In [ ]:
anomaly_scores = list(df_as[['anomaly_score_new', 'good_or_bad', 'a_type_id', 'img_no', 'a_type']].itertuples(index=False, name=None))

## FIND Delta Opt

In [ ]:
delta_opt = ut.Evaluate.find_optimal_separation_threshold(anomaly_scores)
print(f'delta_opt\t:\t{delta_opt}')

In [ ]:
# def generate_html(img_types, results_path, DS_name, anomaly_types, df_as, df, delta_opt, epochs, mask_type, image_counter, results_subs):
#     if isinstance(img_types, list):
#         html_f_name = f'{results_path}/imgs_{DS_name}_{"_".join(img_types)}.html'
#     else:
#         html_f_name = f'{results_path}/imgs_{DS_name}_{img_types}.html'
#     # print('html_f_name', html_f_name)
#     header = '<html><body>'
#     style = """
#     <style>
#         .dotg {height: 30px; width: 30px; background-color: #008000; border-radius: 50%; display: inline-block;}
#         .dotr {height: 30px; width: 30px; background-color: #ff0000; border-radius: 50%; display: inline-block;}
#         table {border: 1px solid black; width: 100%; border-collapse: collapse;}
#         th, td {font-size: 2em; border: 2px solid black; text-align: center; width: 0.13%;} /* Adjust width here */
#         th {background-color: #f2f2f2; position: -webkit-sticky; position: sticky; top: 0; z-index: 1;} /* Sticky header */
#         figcaption {font-size: 1em; border: 1px dotted blue; text-align: left;}
#     </style>
#     """
    
#     method_names = ['Input', 'Recons', 'Anom Map'] + list(np.unique(df.methods)) + ['Gt']
    
#     def get_relative_img_path(a_type, img_id):
#         return f'test_results/{a_type}/{img_id}_{img_types}_{a_type}_{img_id}_{epochs}_{mask_type}.png'

#     with open(html_f_name, 'w') as f:
#         f.write(header)
#         f.write(style)
#         f.write("<table>")
        

#         f.write("<tr>")
#         for method_name in method_names:
#             f.write(f"<th>{method_name}</th>")
#         f.write("</tr>")
        
#         for a_type in anomaly_types:
#             for img_id in range(image_counter.get(a_type, 0)):
#                 results_subpath_cls = os.path.join(results_subs, str(a_type))

#                 local_img_path = os.path.abspath(f'{results_subpath_cls}/{img_id}_{img_types}_{a_type}_{img_id}_{epochs}_{mask_type}.png')
#                 # print(local_img_path)
                
#                 asn = df_as[(df_as.a_type == a_type) & (df_as.img_no == img_id)].anomaly_score_new
#                 asn_bool = asn > delta_opt
#                 pred_type = 'Anomalous' if asn_bool.iloc[0] else 'Good'
                
#                 if a_type != 'good' and os.path.exists(local_img_path):
#                     relative_img_path = get_relative_img_path(a_type, img_id)
#                     dot_class = 'dotg' if asn_bool.iloc[0] else 'dotr'
                    
#                     # Add a new row for each image
#                     f.write("<tr>")
                    
#                     if isinstance(img_types, list):
#                         local_img_pth = os.path.abspath(f'{results_subpath_cls}/{img_id}_{img_types[0]}_{a_type}_{img_id}_{epochs}_{mask_type}.png')
                        
#                         local_iou_pth = os.path.abspath(f'{results_subpath_cls}/{img_id}_{img_types[1]}_{a_type}_{img_id}_{epochs}_{mask_type}.png')
#                         # print()
#                         f.write(f"<td colspan={len(method_names)}><img src='{local_img_pth}' width='100%'>")
#                         f.write("</tr>")
#                         f.write("<tr>")
#                         f.write(f"<td colspan={len(method_names)}><img src='{local_iou_pth}' width='100%'>")

#                     else:
#                         f.write(f"<td colspan={len(method_names)}><img src='{relative_img_path}' width='100%'>")

                        

#                     f.write(f"<figcaption><span class='{dot_class}'></span>&nbsp;{a_type}-{img_id}:&nbsp;&nbsp;{asn.iloc[0]:0.4f}&nbsp;&nbsp;{asn_bool.iloc[0]}-{pred_type}</figcaption></td>")
#                     f.write("</tr>")
        
#         f.write("</table>")
#         f.write('</body></html>')
    
#     print('Data Written in File:', html_f_name)

In [ ]:
import os
import numpy as np
import re



# def natural_sort_key(text):
#     return [int(s) if s.isdigit() else s.lower() for s in re.split(r'(\d+)', text)]

def generate_html(img_types, results_path,exp_type, DS_name, anomaly_types, df_as, df, delta_opt, epochs, mask_type, image_counter, results_subs):
    # Ensure img_types is always a list
    if isinstance(img_types, str):
        img_types = [img_types]

    # Construct HTML file name based on image types
    html_f_name = f'{results_path}/HTML_{exp_type}_{DS_name}_{"_".join(img_types)}.html'
    
    header = "<html><body>"\
                "<div class='main_container'>"\
                    "<div class='hero'>"\
                        "<h1><span>Webpage containing all results [Visual(heatmaps) and Numerical] for </span>"\
                    "<br>Shapley image explanations<br>"\
                    "with data-aware Binary Partition Trees.<br></div>"
                
    style = """
    <style>
    
        .dotg {height: 30px; width: 30px; background-color: #008000; border-radius: 50%; display: inline-block;}
        .dotr {height: 30px; width: 30px; background-color: #ff0000; border-radius: 50%; display: inline-block;}
        table {border: 1px solid black; width: 100%; border-collapse: collapse;}
        th, td {font-size: 2em; border: 2px solid black; text-align: center; width: 0.13%;}
        th {background-color: #f2f2f2; position: -webkit-sticky; position: sticky; top: 0; z-index: 1;}
        figcaption {font-size: 1em; border: 1px dotted blue; text-align: left;}
        .hero {
                    position: relative;
                    background: #333 url(http://srdjanpajdic.com/slike/2.jpg) no-repeat center center fixed;
                    -webkit-background-size: cover;
                    -moz-background-size: cover;
                    background-size: cover;
                    text-align: center;
                    color: #fff;
                    padding-top: 20px;
                    min-height: 300px;
                    letter-spacing: 2px;
                    font-family: "Montserrat", sans-serif;
                    h1 {
                    font-size: 50px;
                    line-height: 1.3;
                    span {
                    font-size: 25px;
                    color: $color2;
                    border-bottom: 2px solid $color2;
                    padding-bottom: 8px;
                    line-height: 3;
                    }
    </style>
    """
    
    # method_names = ['Input', 'Recons', 'Anom Map'] + list(np.unique(df.methods)) + ['Gt']
    # method_names = ['Input', 'Recons', 'Anom Map'] + sorted(np.unique(df.methods), key=natural_sort_key) + ['Gt']
    method_names = ['Input', 'Recons', 'Anom Map'] + list(df.methods.unique()) + ['Gt']
    
    def get_relative_img_path(a_type, img_id, img_type):
        return f'test_results/{a_type}/{img_id}_{img_type}_{a_type}_{img_id}_{epochs}_{mask_type}.png'

    with open(html_f_name, 'w') as f:
        f.write(header)
        f.write(style)
        f.write("<table>")
        
        # Write table headers
        f.write("<tr>")
        for method_name in method_names:
            f.write(f"<th>{method_name}</th>")
        f.write("</tr>")
        
        # Loop through anomaly types and images
        for a_type in anomaly_types:
            for img_id in range(image_counter.get(a_type, 0)):
                results_subpath_cls = os.path.join(results_subs, str(a_type))

                # Get anomaly score
                asn = df_as[(df_as.a_type == a_type) & (df_as.img_no == img_id)].anomaly_score_new
                asn_bool = asn > delta_opt
                pred_type = 'Anomalous' if asn_bool.iloc[0] else 'Good'
                dot_class = 'dotg' if asn_bool.iloc[0] else 'dotr'

                # Check if at least one image exists
                valid_images = []
                for img_type in img_types:
                    local_img_pth = os.path.abspath(f'{results_subpath_cls}/{img_id}_{img_type}_{a_type}_{img_id}_{epochs}_{mask_type}.png')
                    if os.path.exists(local_img_pth):
                        valid_images.append(local_img_pth)

                if a_type != 'good' and valid_images:
                    # Write a row for images
                    for img_path in valid_images:
                        f.write("<tr>")
                        f.write(f"<td colspan={len(method_names)}><img src='{img_path}' width='100%'></td>")
                        f.write("</tr>")

                    # Write anomaly score and classification
                    f.write("<tr>")
                    f.write(f"<td colspan={len(method_names)}><figcaption><span class='{dot_class}'></span>&nbsp;"
                            f"{a_type}-{img_id}:&nbsp;&nbsp;{asn.iloc[0]:0.4f}&nbsp;&nbsp;{asn_bool.iloc[0]}-{pred_type}"
                            f"</figcaption></td>")
                    f.write("</tr>")

        f.write("</table>")
        f.write('</body></html>')
    
    print('Data Written in File:', html_f_name)


In [ ]:
img_types = 'heatmaps'  # or 'IoU'

generate_html(img_types, results_path,exp_type, DS_name, anomaly_types, df_as, df, delta_opt, epochs, mask_type, image_counter, results_subs)


In [ ]:
img_types = 'IoU'  # or 'IoU'
generate_html(img_types, results_path,exp_type, DS_name, anomaly_types, df_as, df, delta_opt, epochs, mask_type, image_counter, results_subs)


In [ ]:
img_types = ['heatmaps','IoU']  # or 'IoU'

generate_html(img_types, results_path,exp_type, DS_name, anomaly_types, df_as, df, delta_opt, epochs, mask_type, image_counter, results_subs)
